# придумаем алгоритм и обучим всё на train-данных

берём данные из train $\Rightarrow$ правильные ответы предобрабатываем (сильно упрощаем) - получаем шаблоны $\Rightarrow$ 


$\Rightarrow$ новый текст будем предобрабатывать и искать подходящий шаблон $\Rightarrow$ восстанавливаем исходный вид текста, соответствующего шаблону

In [214]:
import json

with open('train.json', 'r') as f:
    data = json.load(f)

# списки для хранения текстов и ответов (тип "а" - обеспечение исполнения контракта, тип "b" - обеспечение гарантийных обязательств)
text_a = []
answer_a = []
text_b = []
answer_b = []

for obj in data:
    if obj['label'] == 'обеспечение исполнения контракта':
        text_a.append(obj['text'])
        answer_a.append(obj['extracted_part']['text'][0])
    elif obj['label'] == 'обеспечение гарантийных обязательств':
        text_b.append(obj['text'])
        answer_b.append(obj['extracted_part']['text'][0])

### для 'обеспечение исполнения контракта'

In [215]:
import regex as re
import random
import numpy as np

# предобработаем данные - избавимся от чисел (цифрами и словами), регистров, символов, английских букв и другой встретившейся гадости
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    pattern = r'[\/a-zA-Z]'
    text = re.sub(pattern, '', text)
    text = re.sub(r'[№\'«»…,.–%()_:-]', '', text)
    text = re.sub(r'\u200b', '', text)
    numeral_pattern = r'\b(?:ноль|один|два|три|четыре|пять|шесть|семь|восемь|девять|десять|одиннадцать|двенадцать|тринадцать|четырнадцать|пятнадцать|шестнадцать|семнадцать|восемнадцать|девятнадцать|двадцать|тридцать|сорок|пятьдесят|шестьдесят|семьдесят|восемьдесят|девяносто|сто|двести|триста|четыреста|пятьсот|шестьсот|семьсот|восемьсот|девятьсот|тысяча|тысяч|миллион|пяти|трёх|десяти|двадцати|тридцати|сорока|пятидесяти|шестидесяти|семидесяти|восьмидесяти|девяноста|ста)\b' # да а что
    text = re.sub(numeral_pattern, '', text, flags=re.IGNORECASE)
    text = ' '.join(text.split())
    return text

preprocessed_answers = [preprocess_text(text) for text in answer_a]
preprocessed_texts = [preprocess_text(text) for text in text_a]

# разобьем train:
train_text_size = int(0.9*len(text_a))

train_text_index = random.sample(range(0, len(text_a)), train_text_size)
train_text = np.array(preprocessed_texts)[train_text_index]
train_answers = np.array(preprocessed_answers)[train_text_index]

# теперь все предобработанные правильные ответы из train-датасета находятся в этом ящике:
shablon = set(train_answers)

In [216]:
# будем искать самый длинный шаблон, который встретится в предобработанном валидационном тексте в качестве подстроки
subs = []

test_text_index = list(set(range(len(text_a)))-set(train_text_index))
test_text = np.array(preprocessed_texts)[test_text_index]
test_answers = np.array(preprocessed_answers)[test_text_index]


for text in test_text:
    found_substring = ''
    max_length = 0
    try:
        # определение самой длинной подстроки
        for substring in shablon:
            match = re.search(substring, text)
            if match and len(substring) > max_length:
                found_substring = substring
                max_length = len(substring)
    except:
        print(type(substring))
        break

    subs.append(found_substring)
    


In [217]:
#  найдя шаблон, будем восстанавливать его реальный вид в тексте

prediction = []

text_a_test = np.array(text_a)[test_text_index]
answer_a_test = np.array(answer_a)[test_text_index]

pattern = r'[\/a-zA-Z0-9№«»,.–%()_:\-]'
for i in range(len(test_text)):
    if subs[i] == '':
        prediction.append('')
    else:
        text = text_a_test[i]
        target = r"\b" + (r"\b.+?\b").join(subs[i].split()) + r"\b"  # разобьём шаблон - между словами могут прятаться удалённые символы и числа
        matches = re.findall(target, text, re.IGNORECASE,overlapped=True) # ищем всё, что напоминает разбитый шаблон
    

        if matches:
            min_match = min(matches, key=len) # ищем минимальную подстроку, содержащую реальный вид шаблона
           
            start_index = text.find(min_match)
            end_index = start_index + len(min_match)
            match1 = re.match(r' ' + pattern + '+', text[end_index:])


            # добавим концы строк - если после конца шаблона были символы и числа
            if text[end_index]=='.':
                min_match = min_match + "."    
            else:
                match1 = re.match(r' ' + pattern + '+', text[end_index:])
                if match1:
                    matched_chars = match1.group(0)[1:]
                    min_match = min_match + " " + matched_chars
                match2 = re.match(pattern + '+', text[end_index:])
                if match2:
                    matched_chars = match2.group(0)[0:]
                    min_match = min_match + matched_chars

        else:
            min_match = '' 
        
        
        prediction.append(min_match)


In [221]:
def calculate_accuracy(test_text, predictions):
    matches = sum([1 for i in range(len(test_text)) if test_text[i] == predictions[i]])
    accuracy = matches / len(test_text)
    
    return accuracy


accuracy = calculate_accuracy(test_answers, subs)

print("Как часто брали правильный шаблон: {:.2%}".format(accuracy))

Как часто брали правильный шаблон: 62.63%


In [220]:
accuracy_real = calculate_accuracy(prediction, answer_a_test)
print("Как часто доставали нужную информацию: {:.2%}".format(accuracy_real))

Как часто доставали нужную информацию: 47.47%


### для 'обеспечение исполнения контракта'

In [222]:
preprocessed_answersB = [preprocess_text(text) for text in answer_b]
preprocessed_textsB = [preprocess_text(text) for text in text_b]


train_text_sizeB = int(0.9*len(text_b))

train_text_indexB = random.sample(range(0, len(text_b)), train_text_sizeB)
train_textB = np.array(preprocessed_textsB)[train_text_indexB]
train_answersB = np.array(preprocessed_answersB)[train_text_indexB]

shablonB = set(train_answersB)

In [51]:
subsB = []

test_text_indexB = list(set(range(len(text_b)))-set(train_text_indexB))
test_textB = np.array(preprocessed_textsB)[test_text_indexB]
test_answersB = np.array(preprocessed_answersB)[test_text_indexB]




for text in test_textB:

    found_substring = ''
    max_length = 0
    try:
    
        for substring in shablonB:
            match = re.search(substring, text)
            if match and len(substring) > max_length:
                found_substring = substring
                max_length = len(substring)
    except:
        print(type(substring))
        break
                


    subsB.append(found_substring)
    


In [52]:
text_b_test = np.array(text_b)[test_text_indexB]
answer_b_test = np.array(answer_b)[test_text_indexB]

In [134]:
predictionB = []

text_b_test = np.array(text_b)[test_text_indexB]
answer_b_test = np.array(answer_b)[test_text_indexB]
pattern = r'[\/a-zA-Z0-9№«»,.–%()_:\-]'
for i in range(len(test_textB)):
    if subs[i] == '':
        predictionB.append('')
    else:
        text = text_b_test[i]
        target = r"\b" + (r"\b.+?\b").join(subsB[i].split()) + r"\b"
        matches = re.findall(target, text, re.IGNORECASE,overlapped=True)
    

        if matches:
            min_match = min(matches, key=len)
           
            start_index = text.find(min_match)
            end_index = start_index + len(min_match)
            match1 = re.match(r' ' + pattern + '+', text[end_index:])

            if text[end_index]=='.':
                min_match = min_match + "."    
            else:
                match1 = re.match(r' ' + pattern + '+', text[end_index:])
                if match1:
                    matched_chars = match1.group(0)[1:]
                    min_match = min_match + " " + matched_chars
                match2 = re.match(pattern + '+', text[end_index:])
                if match2:
                    matched_chars = match2.group(0)[0:]
                    min_match = min_match + matched_chars


        
        else:
            min_match = '' 
        
        
        predictionB.append(min_match)


In [226]:
accuracyB = calculate_accuracy(test_answersB, subsB)

print("Как часто брали правильный шаблон: {:.2%}".format(accuracyB))

Как часто брали правильный шаблон: 71.95%


In [223]:
accuracy_realB = calculate_accuracy(predictionB, answer_b_test)
print("Как часто доставали нужную информацию: {:.2%}".format(accuracy_realB))

Как часто доставали нужную информацию: 59.76%


# тест

In [64]:
with open('test.json', 'r') as f:
    data_list = json.load(f)

text_test = []
label = []

for data in data_list:

    text_test.append(data["text"])
    label.append(data["label"])

        

In [83]:
#TEST
preprocessed_test = [preprocess_text(text) for text in text_test]

subs_test = []

for i in range(len(text_test)):
    found_substring = ''
    max_length = 0
    if label[i] == 'обеспечение исполнения контракта':
        for substring in shablon:
            match = re.search(substring, preprocessed_test[i])
            if match and len(substring) > max_length:
                found_substring = substring
                max_length = len(substring)
  
    if label[i] == 'обеспечение гарантийных обязательств':
        for substring in shablonB:
            match = re.search(substring, preprocessed_test[i])
            if match and len(substring) > max_length:
                found_substring = substring
                max_length = len(substring)
    subs_test.append(found_substring)



In [150]:
predictions = []
starts = []
ends = []
pattern = r'[\/a-zA-Z0-9№«»,.–%()_:\-]'
for i in range(len(text_test)):
    if subs_test[i] == '':
        predictions.append('')
        starts.append(0)
        ends.append(0)
    else:
        text = text_test[i]
        target = r"\b" + (r"\b.+?\b").join(subs_test[i].split()) + r"\b"
        matches = re.findall(target, text, re.IGNORECASE,overlapped=True)
    

        if matches:
            min_match = min(matches, key=len)
           
            start_index = text.find(min_match)
            end_index = start_index + len(min_match)
            

            if text[end_index]=='.':
                min_match = min_match + "."    
            else:
                match1 = re.match(r' ' + pattern + '+', text[end_index:])
                if match1:
                    matched_chars = match1.group(0)[1:]
                    min_match = min_match + " " + matched_chars
                match2 = re.match(pattern + '+', text[end_index:])
                if match2:
                    matched_chars = match2.group(0)[0:]
                    min_match = min_match + matched_chars
                
        

        else:
            min_match = '' 

        # считаем начала и концы
        start = text.find(min_match)
        end = start + len(min_match)
        if start==-1:
            starts.append(0)
            ends.append(0)
        else:    
            starts.append(start)
            ends.append(end)
            
        predictions.append(min_match)


In [157]:
# запись в файл

with open('test.json') as f:
  predictions_list=json.load(f)

for i in range(len(predictions_list)):
  predictions_list[i]["extracted_part"] = {
    "text": predictions[i],
    "answer_start": starts[i],
    "answer_end": ends[i]
    }

with open('predictions.json', 'w') as f:
    json.dump(predictions_list, f, ensure_ascii=False)